# Combinator

Following the parameter extraction the filtered data is grouped in bins. These bins are stored in text files with the images that go together. 

This notebook is to develop functions to take these text files and combine the group of images indicated at each one.

In [2]:
import numpy as np
from astropy.io import fits
import pandas as pd
import os
from glob import glob

In [3]:
# Testing files

BINS = "/home/joaopedro/gdrive/Projects/active/Wild_Duck_Pipeline/wd/test_data/test/science/reduced/NGC6067/batches"
IMAGES = "/home/joaopedro/gdrive/Projects/active/Wild_Duck_Pipeline/wd/test_data/test/science/reduced/NGC6067"
TEST_BATCH1 = BINS + "/batch1.txt"

with open(TEST_BATCH1) as test:
    BATCH1 = [i.strip() for i in test]


In [4]:
os.listdir(IMAGES)

['r_OI2019A-031_2019-05-26_0322.fits',
 'r_OI2019A-031_2019-05-26_0310.fits',
 'r_OI2019A-031_2019-05-26_0312.fits',
 'batches',
 'parameters.csv',
 'r_OI2019A-031_2019-05-26_0311.fits',
 'r_OI2019A-031_2019-05-26_0316.fits',
 'r_OI2019A-031_2019-05-26_0315.fits',
 'r_OI2019A-031_2019-05-26_0321.fits',
 'r_OI2019A-031_2019-05-26_0317.fits',
 'r_OI2019A-031_2019-05-26_0320.fits']

In [37]:
# Combine one batch

def combine_batch(batch, update_name):
    """
    From a text file containing FITS files names, combine all images and generate new reference header.
    Give back the new matrix and header. Used from within the folder with the images
    
    Args:
        batch -- List of strings with path to batch text file.
        update_name -- String with the exit name.
    
    Return:
        combination -- 2D numpy array containing combined image
        ref_header -- Astropy header object with updated info.
    """
    # Load images and headers
    
    images = [fits.getdata(file) for file in batch]
    headers = [fits.getheader(file) for file in batch]
    
    #  New parameters
    airmasses = np.array([float(header["AIRMASS"]) for header in headers])
    jds = np.array([float(header["JD"]) for header in headers])
    ncombine = len(images)
    middle = int(ncombine/2)
    date = headers[middle]["DATE-OBS"]
    
    # Generate reference header  
    ref_header = headers[middle]
    ref_header["JD"] = jds.mean()
    ref_header["AIRMASS"] = airmasses.mean()
    ref_header["NCOMBINE"] = ncombine
    ref_header["DATE-OBS"] = date
    for (i, file) in enumerate(batch, start=1):
        ref_header[f"IMCMB{i}"] = file
    ref_header["IMAGE"] = update_name
    
    # Combine images
    
    cube = np.stack(images, axis=0)
    
    combination = np.median(cube, axis=0)
    
    return (combination, ref_header)


# Combine all batches

def combine_batches(images_folder, batches_folder="batches", out_folder="combinated"):
    """
    From a text file containing FITS files names, combine all images and generate new reference header.
    Give back the new matrix and header.
    
    New name definition takes old name (following the conventions) strip the number part and create a new
    based on the combination order.
    
    OBS1: Expects that the batch folder is already created.
    
    Args:
        images_folder -- String with path to the folder with the images.
        batches_folder -- Strings with path to batch folder files relative to the images folder.
        out_folder -- String with the exit folder name relative to .
        
    Return:
        List of strings with path to created files.
        
    File transformations:
        Write new FITS files for the combinations.
    """
    cwd = os.getcwd()
    os.chdir(images_folder)
    
    os.mkdir(out_folder)
    
    # Load Batches
    batches = []
    for batch in os.listdir(batches_folder):
        with open(batches_folder + "/" + batch) as f:
            batches.append([line.strip() for line in f])
            
    # Define new stem name
    ref_file = batches[0][0].split("_")
    stem = f"final_{ref_file[1]}_{ref_file[2]}"
    
    # For each batch combine_batch them save results
    for i, batch in enumerate(batches, start=1):
        new_name = f"{stem}_{i:04}" 
        matrix, new_header = combine_batch(batch, new_name)
        fits.writeto(f"{out_folder}/{new_name}.fits", matrix, header=new_header)
    
    new_fits = os.listdir(f"{images_folder}/{out_folder}")
    new_fits.sort()
    
    os.chdir(cwd)
    
    return new_fits

In [38]:
# Testing

# os.chdir(IMAGES); im, head = combine_batch(BATCH1, "test")  #  OK!! (Need to be run whithin images folder)

images = combine_batches(IMAGES)

In [39]:
type(images)
images

['final_OI2019A-031_2019-05-26_0001.fits',
 'final_OI2019A-031_2019-05-26_0002.fits',
 'final_OI2019A-031_2019-05-26_0003.fits']